# TextGrad Tutorials: Primitives

![TextGrad](https://github.com/vinid/data/blob/master/logo_full.png?raw=true)

An autograd engine -- for textual gradients!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zou-group/TextGrad/blob/main/examples/notebooks/Prompt-Optimization.ipynb)
[![GitHub license](https://img.shields.io/badge/License-MIT-blue.svg)](https://lbesson.mit-license.org/)
[![Arxiv](https://img.shields.io/badge/arXiv-2406.07496-B31B1B.svg)](https://arxiv.org/abs/2406.07496)
[![Documentation Status](https://readthedocs.org/projects/textgrad/badge/?version=latest)](https://textgrad.readthedocs.io/en/latest/?badge=latest)
[![PyPI - Python Version](https://img.shields.io/pypi/pyversions/textgrad)](https://pypi.org/project/textgrad/)
[![PyPI](https://img.shields.io/pypi/v/textgrad)](https://pypi.org/project/textgrad/)

**Objectives for this tutorial:**

* Introduce you to the primitives in TextGrad

**Requirements:**

* You need to have an OpenAI API key to run this tutorial. This should be set as an environment variable as OPENAI_API_KEY.


In [1]:
%pip install -e ../..

Obtaining file:///Users/austintimberlake/Desktop/textgrad
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for textgrad (pyproject.toml) ... done
  Created wheel for textgrad: filename=textgrad-0.1.8-0.editable-py3-none-any.whl size=9866 sha256=4dd67bdbe7dd0c5d87aa9e5aa56280d3f55100b8a62f3821de6b8b063cb31009
  Stored in directory: /private/var/folders/fd/ntt6v6ts64x_yhx3c3054zv00000gn/T/pip-ephem-wheel-cache-43d7hkot/wheels/31/6c/b3/1bcc883beb125269ce5253459fb3951302e0ab797bda886328
Successfully built textgrad
  Attempting uninstall: textgrad
    Found existing installation: textgrad 0.1.8
    Uninstalling textgrad-0.1.8:
      Successfully uninstalled textgrad-0.1.8

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the ke

In [2]:
%pip install textgrad # you might need to restart the notebook after installing textgrad

from textgrad.engine import get_engine
from textgrad import Variable
from textgrad.optimizer import TextualGradientDescent
from textgrad.loss import TextLoss
from dotenv import load_dotenv
load_dotenv()


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

## Introduction: Variable

Variables in TextGrad are the metaphorical equivalent of tensors in PyTorch. They are the primary data structure that you will interact with when using TextGrad. 

Variables keep track of gradients and manage the data.

Variables require two arguments (and there is an optional third one):

1. `data`: The data that the variable will hold
2. `role_description`: A description of the role of the variable in the computation graph
3. `requires_grad`: (optional) A boolean flag that indicates whether the variable requires gradients

In [3]:
x = Variable("A sntence with a typo", role_description="The input sentence", requires_grad=True)

In [4]:
x.gradients

set()

## Introduction: Engine

When we talk about the engine in TextGrad, we are referring to an LLM. The engine is an abstraction we use to interact with the model.

In [5]:
engine = get_engine("gpt-3.5-turbo")

In [6]:
# GPT-5 mini with a custom system prompt and input
engine = get_engine("gpt-4o-mini")
custom_system_prompt = "Evaluate the correctness of this sentence"
custom_input = "The quick brown fox jumps over the lazy dog."
engine.generate(custom_input, system_prompt=custom_system_prompt, temperature=0)

'The sentence "The quick brown fox jumps over the lazy dog." is correct. It is a pangram, meaning it contains every letter of the English alphabet at least once.'

This object behaves like you would expect an LLM to behave: You can sample generation from the engine using the `generate` method. 

In [7]:
engine.generate("Hello how are you?", temperature=0)

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

## Introduction: Loss

Again, Loss in TextGrad is the metaphorical equivalent of loss in PyTorch. We use Losses in different form in TextGrad but for now we will focus on a simple TextLoss. TextLoss is going to evaluate the loss wrt a string.

In [8]:
system_prompt = Variable("Evaluate the correctness of this sentence", role_description="The system prompt")
loss = TextLoss(system_prompt, engine=engine)

## Introduction: Optimizer

Keeping on the analogy with PyTorch, the optimizer in TextGrad is the object that will update the parameters of the model. In this case, the parameters are the variables that have `requires_grad` set to `True`.

**NOTE** This is a text optimizer! It will do all operations with text! 

In [9]:
optimizer = TextualGradientDescent(parameters=[x], engine=engine)


## Putting it all together

We can now put all the pieces together. We have a variable, an engine, a loss, and an optimizer. We can now run a single optimization step.

In [10]:
l = loss(x)
l.backward(engine)
optimizer.step()

In [11]:
x.value

'Notice the typo in this sentence.'

While here it is not going to be useful, we can also do multiple optimization steps in a loop! Do not forget to reset the gradients after each step!

In [12]:
optimizer.zero_grad()

In [13]:
# Chain an LLM call into a TextGrad optimization step
# with a numeric blackbox judge in the loop.

import re
import uuid
from textgrad import BlackboxLLM
from textgrad.autograd import functional as F

polite_system_prompt = Variable(
    "You are a helpful assistant. Respond with a short sentence.",
    role_description="System prompt for generating a sentence",
    requires_grad=True,
)

generate_sentence_prompt = Variable(
    f"Try to say something to get out of an awkward situation. Output only the sentence you want to say. Nonce: {uuid.uuid4()}",
    role_description="User prompt for sentence generation",
    requires_grad=False,
)

model = BlackboxLLM(engine=engine, system_prompt=polite_system_prompt)
raw_sentence = model(generate_sentence_prompt)

judge_engine = get_engine("gpt-4o-mini")

def judge_politeness_score(text: str) -> int:
    judge_prompt = (
        "You are a strict politeness judge. Rate the politeness of the sentence from 1 to 10. "
        "Respond with only a single integer from 1 to 10."
    )
    response = judge_engine.generate(
        f"{judge_prompt}\nSentence: {text}",
        temperature=0,
    )
    match = re.search(r"\d+", response)
    if not match:
        return 1
    score = int(match.group(0))
    return max(1, min(10, score))

# Use a fixed evaluation prompt and optimize the generation system prompt.
# The blackbox judge only provides a numeric score; TextGrad uses it as context.
eval_system_prompt = Variable(
    "You will receive a sentence and a politeness score from a separate blackbox judge. "
    "Explain how to improve the sentence to increase the score toward 10. Be concise and actionable.",
    role_description="The system prompt",
    requires_grad=False,
)
loss = TextLoss(eval_system_prompt, engine=engine)
optimizer = TextualGradientDescent(parameters=[polite_system_prompt], engine=engine)

for step in range(20):
    # Re-generate first so the loss is computed on the current prompt.
    raw_sentence = model(generate_sentence_prompt)

    score = judge_politeness_score(raw_sentence.value)
    score_var = Variable(
        f"Politeness score (1-10) from blackbox judge: {score}",
        role_description="numeric politeness score",
        requires_grad=False,
    )

    loss_input = F.sum(
        [
            Variable("Sentence:\n", role_description="loss prefix", requires_grad=False),
            raw_sentence,
            Variable("\n", role_description="loss separator", requires_grad=False),
            score_var,
        ]
    )

    l = loss(loss_input)
    l.backward(engine)
    optimizer.step()
    optimizer.zero_grad()

    print(f"Step {step + 1} system prompt:")
    print(polite_system_prompt.value)
    print("LLM output:")
    print(raw_sentence.value)
    print(f"Blackbox score: {score}")
    print("-" * 40)


KeyboardInterrupt: 

In [2]:
import sys
import os

# Add the subspace-rerouting submodule to the path (relative to this notebook)
# Notebook is in examples/notebooks/, submodule is in external/subspace-rerouting
submodule_path = os.path.abspath(os.path.join(os.getcwd(), '../../external/subspace-rerouting'))
sys.path.append(submodule_path)

# Install required dependencies for the subspace-rerouting submodule
%pip install rich python-dotenv scikit-learn toml transformer-lens

from ssr.lens import Lens

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached anyio-4.12.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 k

/Users/brian/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Example: Using Lens to get activations and feed them to a layer 11 classifier

# ============================================================================
# Authenticate with Hugging Face (required for gated models like Llama)
# ============================================================================
# Load .env file from subspace-rerouting submodule
from dotenv import load_dotenv
import os

# Load .env file from the submodule directory
submodule_path = os.path.abspath(os.path.join(os.getcwd(), '../../external/subspace-rerouting'))
env_path = os.path.join(submodule_path, '.env')
load_dotenv(env_path)

# Authenticate with Hugging Face using token from .env
try:
    from huggingface_hub import login
    
    # Get token from environment (loaded from .env file)
    hf_token = os.getenv('HF_TOKEN')
    
    if hf_token:
        login(token=hf_token)
        print("✓ Authenticated with Hugging Face using token from .env file")
    else:
        print("⚠️  HF_TOKEN not found in .env file")
        print("   Please add HF_TOKEN=your_token_here to the .env file")
        print("   Get your token from: https://huggingface.co/settings/tokens")
        print("   Request access to model: https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct")
        # Try to login interactively as fallback
        login()
except ImportError:
    print("⚠️  huggingface_hub not installed. Install with: pip install huggingface_hub")
    print("   Or set HF_TOKEN environment variable with your Hugging Face token")

# ============================================================================
# Initialize Lens with a preset model (e.g., llama3.2_1b)
# ============================================================================
# Available presets: "llama3.2_1b", "llama3.2_3b", "gemma2_2b", "qwen2.5_1.5b"
lens = Lens.from_preset("llama3.2_1b")

# Define your user message (no system prompt)
user_message = "What is the capital of France?"

# Apply chat template to format the message properly (without system prompt)
formatted_prompt = lens.apply_chat_template(
    user_message,
    add_generation_prompt=True  # Add generation prompt
)

print("Formatted prompt:")
print(formatted_prompt)
print("\n" + "="*60 + "\n")

# Tokenize the formatted prompt
tokens = lens.apply_chat_template(
    user_message,
    tokenize=True,
    add_generation_prompt=True
)

print(f"Tokenized input shape: {tokens.input_ids.shape}")
print(f"Tokens: {lens.model.to_str_tokens(tokens.input_ids[0])}")
print("\n" + "="*60 + "\n")

# Run the model with cache to get activations
# This returns logits and an ActivationCache containing all intermediate activations
logits, cache = lens.model.run_with_cache(
    tokens.input_ids,
    names_filter=lambda name: "resid_post" in name  # Filter to get residual stream activations
)

# Extract activations from layer 11
layer_idx = 11
layer_activations = cache["resid_post", layer_idx]
print(f"Layer {layer_idx} activations shape: {layer_activations.shape}")
print(f"  - Batch size: {layer_activations.shape[0]}")
print(f"  - Sequence length: {layer_activations.shape[1]}")
print(f"  - Hidden dimension: {layer_activations.shape[2]}")

# Get activations at the last token position (often used for classification/analysis)
last_token_activations = layer_activations[:, -1, :]  # Shape: (batch_size, hidden_dim)
print(f"\nLast token activations shape: {last_token_activations.shape}")

# ============================================================================
# Load and use the layer 11 classifier
# ============================================================================
print("\n" + "="*60)
print("Loading Layer 11 Classifier:")
print("="*60)

import joblib
import numpy as np
from pathlib import Path
import os

# Get the path to the subspace-rerouting submodule
submodule_path = os.path.abspath(os.path.join(os.getcwd(), '../../external/subspace-rerouting'))
classifier_dir = Path(submodule_path) / 'mlp_classifiers'

# Find the most recent metadata file (or specify a specific one)
metadata_files = sorted(classifier_dir.glob('*_metadata.pkl'))

if len(metadata_files) == 0:
    raise ValueError(f"⚠️  No saved classifiers found in {classifier_dir}")
    
# Load the most recent classifier set
metadata_path = metadata_files[-1]
print(f"\nLoading classifiers from: {metadata_path.name}")

# Load metadata
metadata = joblib.load(metadata_path)
print(f"  Timestamp: {metadata['timestamp']}")
print(f"  Layers trained: {metadata['layers_to_train']}")
print(f"  Features per layer: {metadata['n_features']}")

# Extract base path from metadata filename
base_name = metadata_path.stem.replace('_metadata', '')

# Load layer 11 classifier
layer_11_path = classifier_dir / f'{base_name}_layer_{layer_idx}.pkl'

if not layer_11_path.exists():
    raise ValueError(f"⚠️  Layer {layer_idx} classifier not found at {layer_11_path}")

layer_11_data = joblib.load(layer_11_path)
mlp_classifier = layer_11_data['mlp']
scaler = layer_11_data['scaler']

print(f"\n✓ Loaded layer {layer_idx} classifier")
print(f"  Test accuracy: {layer_11_data['test_accuracy']:.4f}")
print(f"  Train accuracy: {layer_11_data['train_accuracy']:.4f}")
print(f"  Label names: {layer_11_data.get('label_names', 'N/A')}")

# ============================================================================
# Feed activations to the classifier
# ============================================================================
print("\n" + "="*60)
print("Classifying Activations:")
print("="*60)

# Convert activations to numpy and prepare for classification
# Use last token activations (shape: batch_size, hidden_dim)
activations_np = last_token_activations.detach().cpu().numpy()

# Standardize using the same scaler used during training
activations_scaled = scaler.transform(activations_np)

# Get predictions
predictions = mlp_classifier.predict(activations_scaled)
prediction_proba = mlp_classifier.predict_proba(activations_scaled)

print(f"\nInput activations shape: {activations_np.shape}")
print(f"Scaled activations shape: {activations_scaled.shape}")
print(f"\nPredicted class: {predictions[0]}")

# Print probabilities for each class
if 'label_names' in layer_11_data:
    label_names = layer_11_data['label_names']
    print(f"\nClass probabilities:")
    for i, (label, prob) in enumerate(zip(label_names, prediction_proba[0])):
        print(f"  {label}: {prob:.4f}")
else:
    print(f"\nClass probabilities:")
    for i, prob in enumerate(prediction_proba[0]):
        print(f"  Class {i}: {prob:.4f}")

print(f"\n✓ Classification complete!")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✓ Authenticated with Hugging Face using token from .env file


`torch_dtype` is deprecated! Use `dtype` instead!


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:40                                                                                   │
│                                                                                                  │
│    37 # Initialize Lens with a preset model (e.g., llama3.2_1b)                                  │
│    38 # ============================================================================             │
│    39 # Available presets: "llama3.2_1b", "llama3.2_3b", "gemma2_2b", "qwen2.5_1.5b"             │
│ ❱  40 lens = Lens.from_preset("llama3.2_1b")                                                     │
│    41                                                                                            │
│    42 # Define your user message (no system prompt)                                              │
│    43 user_message = "What is the capital of France?"                                            │
│                                                                                                  │
│ /Users/brian/Desktop/TextGrad-clone/external/subspace-rerouting/ssr/lens.py:140 in from_preset   │
│                                                                                                  │
│   137 │   │   │   │   device=default_values.device,                                              │
│   138 │   │   │   │   dtype="float16",                                                           │
│   139 │   │   │   │   center_unembed=True,                                                       │
│ ❱ 140 │   │   │   │   center_writing_weights=True,                                               │
│   141 │   │   │   │   fold_ln=True,                                                              │
│   142 │   │   │   )                                                                              │
│   143 │   │   else:                                                                              │
│                                                                                                  │
│ /Users/brian/miniconda3/lib/python3.13/site-packages/transformer_lens/HookedTransformer.py:1417  │
│ in from_pretrained_no_processing                                                                 │
│                                                                                                  │
│   1414 │   │   Wrapper for from_pretrained with all boolean flags related to simplifying the mo  │
│   1415 │   │   False. Refer to from_pretrained for details.                                      │
│   1416 │   │   """                                                                               │
│ ❱ 1417 │   │   return cls.from_pretrained(                                                       │
│   1418 │   │   │   model_name,                                                                   │
│   1419 │   │   │   fold_ln=fold_ln,                                                              │
│   1420 │   │   │   center_writing_weights=center_writing_weights,                                │
│                                                                                                  │
│ /Users/brian/miniconda3/lib/python3.13/site-packages/transformer_lens/HookedTransformer.py:1370  │
│ in from_pretrained                                                                               │
│                                                                                                  │
│   1367 │   │                                                                                     │
│   1368 │   │   # Get the state dict of the model (ie a mapping of parameter names to tensors),   │
│   1369 │   │   # match the HookedTransformer parameter names.                                    │
│ ❱ 1370 │   │   state_dict = loading.get_pretrained_state_dict(                                   │
│   1371 │   │   │   official_model_name, cfg, hf_model, dtyp